In [7]:
import requests
import os
import datetime
import zipfile
from tqdm import tqdm
import pandas as pd

class DataProcessor:
    def __init__(self, base_url, zip_download_dir, csv_extract_dir):
        self.base_url = base_url
        self.zip_download_dir = zip_download_dir
        self.csv_extract_dir = csv_extract_dir

    def _download_zip_file(self, link, filename):
        # Check if the file already exists
        if os.path.exists(filename):
            print(f"File {filename} already exists. Skipping download.")
            return True

        response = requests.get(link)
        if response.status_code == 200:
            with open(filename, 'wb') as file:
                file.write(response.content)
            return True
        else:
            print(f"Error: {response.status_code} - Unable to download the ZIP file for {filename}")
            return False

    def _extract_zip_file(self, zip_path):
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            for file_info in zip_ref.infolist():
                # Construct the full path for the extracted file
                extracted_file_path = os.path.join(self.csv_extract_dir, file_info.filename)

                # Check if the file already exists, and skip extraction if it does
                if os.path.exists(extracted_file_path):
                    print(f"File {extracted_file_path} already exists. Skipping extraction.")
                    continue

                # Extract the file
                zip_ref.extract(file_info, path=self.csv_extract_dir)

    def download_and_extract_data(self, start_date, end_date):
        os.makedirs(self.zip_download_dir, exist_ok=True)
        os.makedirs(self.csv_extract_dir, exist_ok=True)
        download_bar = tqdm(total=(end_date - start_date).days + 1, desc="Downloading and Extracting")
        
        for current_date in (start_date + datetime.timedelta(n) for n in range((end_date - start_date).days + 1)):
            date_str = current_date.strftime('%Y-%m-%d')
            link = f'{self.base_url}ETHUSDT-aggTrades-{date_str}.zip'
            filename = os.path.join(self.zip_download_dir, f'ETHUSDT-aggTrades-{date_str}.zip')

            if self._download_zip_file(link, filename):
                self._extract_zip_file(filename)
            download_bar.update(1)
        download_bar.close()

    def process_csv_files(self):
        csv_files = [file for file in os.listdir(self.csv_extract_dir) if file.endswith('.csv')]
        csv_files.sort()
        combined_df = None
        progress_bar = tqdm(total=len(csv_files), desc="Combining CSV files")
        
        for csv_file in csv_files:
            file_path = os.path.join(self.csv_extract_dir, csv_file)
            df = pd.read_csv(file_path)
            if combined_df is None:
                combined_df = df
            else:
                combined_df = pd.concat([combined_df, df])
            progress_bar.update(1)
        
        progress_bar.close()
        return combined_df

    def _convert_to_seconds(self, df):
        df['utc_time'] = pd.to_datetime(df['transact_time'], unit='ms')
        df.set_index('utc_time', inplace=True)
        
        resampled_df = df.resample('S').agg({
            'agg_trade_id': 'first',
            'price': 'ohlc',
            'quantity': 'sum',
            'first_trade_id': 'first',
            'last_trade_id': 'last',
            'is_buyer_maker': 'last',
            # 'transact_time': 'last',
        })
        resampled_df.reset_index(inplace=True)
        resampled_df.columns = resampled_df.columns.droplevel()
        resampled_df.fillna(method='ffill', inplace=True)
        resampled_df.fillna(method='bfill', inplace=True)
        
        return resampled_df

    def run_data_processing_workflow(self, start_date, end_date):
        self.download_and_extract_data(start_date, end_date)
        df = self.process_csv_files()
        df = self._convert_to_seconds(df)
        # define start and end date of df
        # df = df[(df['utc_time'] >= start_date) & (df['utc_time'] <= end_date)]
        return df

    def combine_select_csv_to_df(self, start_date, end_date):
        csv_files = [file for file in os.listdir(self.csv_extract_dir) if file.endswith('.csv')]
        csv_files.sort()
        combined_df = None
        progress_bar = tqdm(total=len(csv_files), desc="Combining CSV files")

        for csv_file in csv_files:
            # Extract the date part from the file name and convert it to a date
            date_str = '-'.join(csv_file.split('-')[-3:]).replace('.csv', '')
            file_date = datetime.datetime.strptime(date_str, '%Y-%m-%d').date()

            # Check if the file date is within the specified date range
            if start_date <= file_date <= end_date:
                file_path = os.path.join(self.csv_extract_dir, csv_file)
                df = pd.read_csv(file_path)
                df = self._convert_to_seconds(df)  # Apply _convert_to_seconds
                combined_df = pd.concat([combined_df, df]) if combined_df is not None else df
            progress_bar.update(1)

        progress_bar.close()
        return combined_df



    def delete_all_data(self):
        os.system(f'rm -rf {self.zip_download_dir}')
        os.system(f'rm -rf {self.csv_extract_dir}')

# Example usage:
base_url = 'https://data.binance.vision/data/futures/um/daily/aggTrades/ETHUSDT/'
zip_download_dir = '/allah/freqtrade/json_dict/binance_aggTrades'
csv_extract_dir = '/allah/freqtrade/json_dict/decompressed_csv'

data_processor = DataProcessor(base_url, zip_download_dir, csv_extract_dir)
start_date = datetime.date(2023, 9, 15)
end_date = datetime.date(2023, 10, 1)

# df = data_processor.run_data_processing_workflow(start_date, end_date)
# df = data_processor.combine_select_csv_to_df(datetime.date(2022, 9, 13), datetime.date(2023, 1, 1))
df = data_processor.run_data_processing_workflow(datetime.date(2022, 9, 13), datetime.date(2023, 1, 1))


Combining CSV files: 100%|██████████| 62/62 [00:01<00:00, 51.16it/s] 


In [25]:
current_timestamp = pd.Timestamp('2023-09-16 23:59:59')

# Iterate through the DataFrame in reverse order and modify timestamps
for index, row in df.iterrows():
    df.at[index, 'timestamp'] = current_timestamp
    current_timestamp -= pd.Timedelta(minutes=1)

# Display the modified DataFrame
print(df)

                            agg_trade_id     open     high      low    close  \
0      2023-09-13 00:00:04  1.378371e+09  1592.29  1592.29  1592.01  1592.01   
1      2023-09-13 00:00:05  1.378371e+09  1592.02  1592.02  1592.01  1592.01   
2      2023-09-13 00:00:06  1.378371e+09  1592.02  1592.33  1592.01  1592.33   
3      2023-09-13 00:00:07  1.378371e+09  1592.24  1592.25  1592.13  1592.14   
4      2023-09-13 00:00:08  1.378371e+09  1592.12  1592.13  1592.08  1592.08   
...                    ...           ...      ...      ...      ...      ...   
345583 2023-09-16 23:59:55  1.380247e+09  1633.68  1633.68  1633.68  1633.68   
345584 2023-09-16 23:59:56  1.380247e+09  1633.68  1633.68  1633.68  1633.68   
345585 2023-09-16 23:59:57  1.380247e+09  1633.68  1633.68  1633.68  1633.68   
345586 2023-09-16 23:59:58  1.380247e+09  1633.68  1633.68  1633.68  1633.68   
345587 2023-09-16 23:59:59  1.380247e+09  1633.68  1633.68  1633.68  1633.68   

        quantity  first_trade_id  last_